In [1]:
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from bokePolicy import PolicyNet, NinebyNineGames
%matplotlib inline

In [4]:
data = NinebyNineGames("boards.csv", scale = 100)
dataloader = DataLoader(data, batch_size = 32, shuffle = True)

pi = PolicyNet()
err = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(pi.parameters(), lr = 0.001)

In [3]:
data[0]

(tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.]],
 
         [[2., 2., 2., 2., 2., 2., 2., 2., 2.],
          [2., 2., 2., 2., 2., 2., 2., 2., 2.],
          [2., 2., 2., 2., 2., 2.,

'7'

In [ ]:
#GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
pi.to(device)

epochs = 1
for _ in range(epochs):
    running_loss = 0.0
    for i, data in tqdm(enumerate(dataloader,0)):
        inputs, moves = data
        inputs, moves = inputs.to(device), moves.to(device)
        
        optimizer.zero_grad()
        outputs = pi(inputs)
        
        #backprop
        loss = err(outputs, moves)
        loss.backward()
        optimizer.step()
    
        running_loss += loss.item()
        if i % 10000 == 9999 :
            print(f"Epoch: {epoch+1}, Loss: {running_loss:.3f}")
            running_loss = 0.0


In [ ]:
torch.save(pi.state_dict(), r"/home/jupyter/BokeGo/policy_train_1.pt")